In [3]:
# -*- coding: utf-8 -*-
"""
Created on Mon Dec 23 15:07:09 2019

"""
import pandas as pd
from sklearn.metrics import f1_score
import numpy as np


def optF1Score( pt,y ):
    '''
    # 针对macro-f1进行结果的优化
    ool = [N_trainSample,N_label], N_trainSample为训练集样本个数，N_label为标签个数
    eg:
        ool = array(
        [[0.49505357, 0.08540789, 0.41953854],
        [0.39238397, 0.21128543, 0.3963306 ],
        [0.45673946, 0.12019312, 0.42306742],
        [0.57948197, 0.07797233, 0.34254569]])
    y: [N_trainSample,1]
    eg: [1,2,1,0]
    
    '''
    from scipy import optimize
    
    pt = np.array(pt)
    def fun(x):
        tmp = np.hstack(
                [x[0] * pt[:, 0].reshape(-1, 1), 
                 x[1] * pt[:, 1].reshape(-1, 1), 
                 x[2] * pt[:, 2].reshape(-1, 1)])
        return - f1_score( np.array(y), np.argmax(tmp, axis=1), average='macro' )
    initW = np.asarray((1,1,1))
    
    return optimize.fmin_powell(fun, initW)


In [4]:
oof_train = pd.read_csv('cache/opt-f1score.csv')
pWeight = optF1Score( oof_train[['p1', 'p2', 'p3']],oof_train['y'] )

Optimization terminated successfully.
         Current function value: -0.733862
         Iterations: 3
         Function evaluations: 303


In [5]:
oof_train.head()

,p1,p2,p3,y
0,-0.941132,0.461139,-0.294789,1
1,-1.674449,1.630520,-1.304830,1
2,-2.581116,2.886493,-1.065741,1
3,-3.278459,-0.212601,2.951491,2
4,-2.658640,1.836897,-0.116195,1


In [6]:
pWeight

array([1.21294424, 0.87312334, 1.11360587])

In [ ]:
# -*- coding:utf-8 -*-
import pandas as pd
import numpy as np
import re
import tensorflow as tf
from tqdm import tqdm
from modeling import Classification
from bert import tokenization
import os

In [ ]:

def convert_single_example( tokenizer, text_a ):

  tokens_a = tokenizer.tokenize(text_a)
  tokens = []
  tokens.append("[CLS]")
  for token in tokens_a:
    tokens.append(token)
  tokens.append("[SEP]")

  input_ids = tokenizer.convert_tokens_to_ids(tokens)# 将中文转换成ids
  input_mask = [1] * len(input_ids) # 创建mask

  return tokens,input_ids,input_mask # 对应的就是创建bert模型时候的input_ids,input_mask,segment_ids 参数

def process(data, args, tokenizer):
    # 数据预处理
    T1 = []
    T2 = []
    for d in tqdm(data.iterrows()):
        d = d[1]
        text = str(d['微博中文内容'])
        text = text[-(args.max_x_length-2):]
        tokens,t1,t2 = convert_single_example( tokenizer, text )
        T1.append( list(t1)+[0] * (args.max_x_length - len(list(t1))) )
        T2.append( list(t2)+[0] * (args.max_x_length - len(list(t2))) )
        
    return T1,T2

def predict( T1,T2, model ):
    rs = []
    bs = 200
    num = int( len(T1)/bs ) + 1
    for n in tqdm(range(num)):
        T1_ = T1[n*bs:(n+1)*bs]
        T2_ = T2[n*bs:(n+1)*bs]
        if len(T1_)>0:
            feed = { model.t1_:T1_, model.t2_:T2_ }
            _c = sess.run([model.logits], feed_dict=feed)[0]
            rs.extend(_c)
    return np.array(rs)


In [ ]:
class Config(object):
    
    batchSize = 32
    max_x_length = 140
    learning_rate = 0.00002

In [ ]:

args = Config()

output_path = '../submit/Result_0326_1.csv'

bert_vocab_file = './chinese_roberta_wwm_ext_L-12_H-768_A-12/vocab.txt'
tokenizer = tokenization.FullTokenizer( vocab_file=bert_vocab_file, do_lower_case=False)

test_reviews = pd.read_excel('../data/nCov_10k_test.xlsx').fillna('')
T1,T2 = process(test_reviews, args, tokenizer)

k_folds = 5
for mode in range(k_folds):

    print ('\n',mode+1)
    if mode>0:
        tf.reset_default_graph()
    model = Classification( args )
    model_dir = './model_saved/{}'.format(mode + 1)
    sess = tf.Session()
    ckpt = tf.train.get_checkpoint_state(model_dir)
    if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
        model.saver.restore(sess, ckpt.model_checkpoint_path)
    else:
        raise ValueError('No such file:[{}]'.format(model_dir))

    c_ = predict( T1,T2, model )

    if mode==0:
        rs_c=c_
    else:
        rs_c+=c_

rs_c = rs_c/k_folds*pWeight
cc_ = np.argmax(rs_c,1)

submit = test_reviews[['微博id']]
submit['id'] = submit['微博id']
del submit['微博id']
submit['y'] = [i-1 for i in cc_]
submit[['id','y']].to_csv(output_path, encoding='utf-8', index=None)
